In [1]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5
!pip install pybullet
!pip install GitPython

!pip install stable_baselines3 -qU

  Using cached pybullet-3.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached pybullet-3.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (103.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 20.0 MB/s eta 0:00:00


In [2]:
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [3]:
!git clone https://github.com/Gaianeve/gym-PIP.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'gym-PIP'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 86 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 25.07 KiB | 1.93 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [4]:
!pip install /content/gym-PIP

Processing ./gym-PIP
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [5]:
# Enter the environment directory
%cd /content/gym-PIP

import gym
# Importing environment
import gym_PIP


/content/gym-PIP


In [6]:
import logging
import time
from pathlib import Path
from typing import Optional, Union

import gym
from gym.spaces import Box
import numpy as np


class HistoryWrapper(gym.Wrapper):
    """Track history of observations for a given number of steps. Initial steps are zero-filled."""

    def __init__(self, env, steps, beta, use_continuity_cost):
        super().__init__(env)
        assert steps > 1, "steps must be > 1"
        self.steps = steps
        self.use_continuity_cost = use_continuity_cost
        self.beta = beta  # weight of continuity cost

        # Initialize step_low and step_high
        self.step_low = np.concatenate([self.observation_space.low, self.action_space.low])
        self.step_high = np.concatenate([self.observation_space.high, self.action_space.high])

        # Stack for each step
        obs_low = np.tile(self.step_low, (self.steps, 1))
        obs_high = np.tile(self.step_high, (self.steps, 1))

        self.observation_space = Box(low=obs_low.flatten(), high=obs_high.flatten())

        self.history = self._make_history()

    def _make_history(self):
        return [np.zeros_like(self.step_low) for _ in range(self.steps)]

    def _continuity_cost(self, obs):
        action = obs[-1][-self.action_space.shape[0]:]
        last_action = obs[-2][-self.action_space.shape[0]:]
        continuity_cost = np.power((action - last_action), 2).sum()
        return continuity_cost

    def step(self, action):
        obs, reward, done, info = self.env.step(action)

        #check obs are in the right form for concatenation
        if obs.shape != (12,):
          obs = obs.ravel()

        self.history.pop(0)

        obs = np.concatenate([obs, action])
        self.history.append(obs)
        obs = np.array(self.history, dtype=np.float32)

        if self.use_continuity_cost:
            continuity_cost = self._continuity_cost(obs)
            reward -= self.beta * continuity_cost
            info["continuity_cost"] = continuity_cost

        return obs.flatten(), reward, done, info

    def reset(
    self,
    seed: Optional[int] = None,
    options: Optional[dict] = None,
    ):
      # Chiama il reset dell'ambiente per ottenere l'osservazione iniziale
      obs = self.env.reset(seed=seed, options=options)

      # Assicurati che obs abbia la forma corretta (aggiungi zeri per le azioni)
      obs = np.concatenate([
          obs,
          np.zeros_like(self.env.action_space.low)
      ])

      # Imposta self.history con la forma corretta
      self.history = [np.zeros_like(obs) for _ in range(self.steps)]
      self.history[0] = obs  # Imposta la prima osservazione

      # Restituisce la storia come array appiattito
      return np.array(self.history, dtype=np.float32).flatten()

In [7]:

# making the environment
def make_env(gym_id, seed, idx, capture_video, run_name, beta):
    def thunk():
        print('Dissennatore')
        env = gym.make(gym_id)
        print('done environment')
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env = HistoryWrapper(env, 4, beta, True)
         # Add this check after creating the env instance
        if not hasattr(env, 'pippa_id'):  # Or another suitable check for successful model loading
            raise ValueError("Robot model not properly loaded in the environment.")
        if capture_video:
            if idx == 0:
              #record video every 500 episodes
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}", \
                                               episode_trigger = lambda x: x % 500 == 0)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk

"""## Create parallel environments  🌍 🦐"""

# vectorize environment
def vectorize_env(gym_id, seed, capture_video, run_name, num_envs, beta):
  envs = gym.vector.SyncVectorEnv(
        [make_env(gym_id, seed + i, i, capture_video, run_name, beta) for i in range(num_envs)]
  )
  assert isinstance(envs.single_action_space, gym.spaces.Box), \
  "only continuous action space as Box is supported"
  return envs

In [8]:
# -*- coding: utf-8 -*-
"""gSDE_class.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/14-PmUmqjlT_F3frLO4-ootagzOeJ1EZS
"""

from typing import Any, Dict, List, Optional, Tuple, TypeVar, Union

import numpy as np
import torch as th
from gym import spaces
from torch import nn

## Needed class for gSDE 🎡
"""
The **TanhBijector** class defines a bijective transformation using the hyperbolic tangent function (tanh). This class is often used in reinforcement learning algorithms to squash the output of the policy network to ensure that the actions remain within a specific range, typically [-1,1]
"""

from stable_baselines3.common.distributions import Distribution, TanhBijector, sum_independent_dims
from torch.distributions import Bernoulli, Categorical, Normal

"""# State Dependent Noise Distribution (gSDE) 🌵
🪄 Distribution class for using generalized State Dependent Exploration (gSDE).

Paper: https://arxiv.org/abs/2005.05719 🦄

It is used to create the noise exploration matrix and compute the log probability of an action with that noise.

   * :`param action_dim`: Dimension of the action space.
   * :`param full_std:` Whether to use (n_features x n_actions) parameters for the std instead of only (n_features,)
   * :`param use_expln:` Use `expln()` function instead of `exp()` to ensure a positive standard deviation (cf paper). It allows to keep variance above zero and prevent it from growing too fast. In practice, `exp()` is usually enough.
   * `:param squash_output`: Whether to squash the output using a tanh function, this ensures bounds are satisfied.

   * `:param learn_features`: Whether to learn features for gSDE or not. This will enable gradients to be backpropagated through the features ``latent_sde`` in the code.

   * `:param epsilon:` small value to avoid NaN due to numerical imprecision.
"""


"""## gSDE class 🐧"""

class gSDE(Distribution):
    bijector: Optional[TanhBijector]
    weights_dist: Normal
    exploration_mat: th.Tensor
    exploration_matrices: th.Tensor

    def __init__(
        self,
        action_dim: int,
        observation_dim: int,
        observation: th.Tensor,
        mean_actions: th.Tensor,
        log_std: th.Tensor,
        full_std: bool = True,
        use_expln: bool = False,
        squash_output: bool = False,
        learn_features: bool = True,
        epsilon: float = 1e-6,
    ):
        super().__init__()
        self.action_dim = action_dim
        self.obs_dim = observation_dim
        self.x = observation
        self.mean_actions = mean_actions
        self.log_std = log_std
        self.use_expln = use_expln
        self.full_std = full_std
        self.epsilon = epsilon
        self.learn_features = learn_features
        self.bijector = TanhBijector(epsilon) if squash_output else None

        #get combination of action and coordinates for noise computation
        # Linear layer to output flattened dimensions
        self.latent_sde = th.nn.Linear(self.obs_dim,1)
        self._latent_sde = self.latent_sde(self.x)
        self.latent_sde_dim = self._latent_sde.shape

        #initializing distribution of actions
        self.distribution = self.proba_distribution()

    #-------------------------------------------- get actions ------------------------------------------------------
    def get_std(self) -> th.Tensor:
        """
        Get the standard deviation from the learned parameter
        (log of it by default). This ensures that the std is positive.

        :param log_std:
        :return:
        """
        if self.use_expln:
            # From gSDE paper, it allows to keep variance
            # above zero and prevent it from growing too fast
            below_threshold = th.exp(self.log_std) * (self.log_std <= 0)
            # Avoid NaN: zeros values that are below zero
            self.safe_log_std = self.log_std * (self.log_std > 0) + self.epsilon
            above_threshold = (th.log1p(self.safe_log_std) + 1.0) * (self.log_std > 0)
            std = below_threshold + above_threshold
        else:
            # Use normal exponential
            std = th.exp(self.log_std)

        if self.full_std:
            return std
        assert self.latent_sde_dim is not None
        # Reduce the number of parameters:
        return th.ones(self.latent_sde_dim, self.action_dim).to(self.log_std.device) * std



    def sample_weights(self, batch_size: int = 1)-> th.Tensor:
      """
      Sample weights for the noise exploration matrix,
      using a centered Gaussian distribution.

      :param log_std:
      :param batch_size:
      """
      std = self.get_std()
      self.weights_dist = Normal(th.zeros_like(std), std)
      # Reparametrization trick to pass gradients
      self.exploration_mat = self.weights_dist.rsample()
      # Pre-compute matrices in case of parallel exploration
      exploration_matrices = self.weights_dist.rsample((batch_size,))
      return exploration_matrices

    def get_noise(self, batch_size: int = 1) -> th.Tensor:
        self.exploration_matrices = self.sample_weights()
        self._latent_sde = self._latent_sde if self.learn_features else self._latent_sde.detach()
        if len(self._latent_sde) == 1 or len(self._latent_sde) != len(self.exploration_mat):
            return self._latent_sde * self.exploration_mat
        noise = self._latent_sde * self.exploration_mat
        return noise


    def proba_distribution(self) -> Normal:
        """
        Create the distribution given its parameters (mean, std)

        :param mean_actions:
        :param log_std:
        :param latent_sde:
        :return:
        """
        # Stop gradient if we don't want to influence the features
        self._latent_sde = self._latent_sde if self.learn_features else self._latent_sde.detach()
        variance = self._latent_sde**2*self.get_std() ** 2
        distribution = Normal(self.mean_actions, th.sqrt(variance + self.epsilon))
        return distribution

    #get action
    def sample(self) -> th.Tensor:
        noise = self.get_noise()
        self.distribution = self.proba_distribution()
        actions = self.distribution.mean + noise
        if self.bijector is not None:
            return self.bijector.forward(actions)
        return actions

# --------------------------------------------- logprobs and entropy --------------------------------------------
#to be both called after self.sample, otherwise self.distribution wuold be Nan and you get nothing

    def log_prob(self, actions: th.Tensor) -> th.Tensor:
        if self.bijector is not None:
            gaussian_actions = self.bijector.inverse(actions)
        else:
            gaussian_actions = actions
        # log likelihood for a gaussian
        log_prob = self.distribution.log_prob(gaussian_actions)
        # Sum along action dim
        log_prob = sum_independent_dims(log_prob)

        if self.bijector is not None:
            # Squash correction (from original SAC implementation)
            log_prob -= th.sum(self.bijector.log_prob_correction(gaussian_actions), dim=1)
        return log_prob

    def entropy(self) -> Optional[th.Tensor]:
        if self.bijector is not None:
            # No analytical form,
            # entropy needs to be estimated using -log_prob.mean()
            return None
        return sum_independent_dims(self.distribution.entropy())


# ---------------------------------------------- auxiliary functions  ------------------------------------------------
#not stricly needed for my PPO. Might still be useful, so keep them
    def mode(self) -> th.Tensor:
        actions = self.distribution.mean
        if self.bijector is not None:
            return self.bijector.forward(actions)
        return actions

    def actions_from_params(
        self, mean_actions: th.Tensor, deterministic: bool = False
    ) -> th.Tensor:
        # Update the proba distribution
        self.proba_distribution(mean_actions, self.log_std)
        return self.get_actions(deterministic=deterministic)

    def log_prob_from_params(
        self, mean_actions: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        actions = self.actions_from_params(mean_actions)
        log_prob = self.log_prob(actions)
        return actions, log_prob

    def proba_distribution_net(self):
        pass

In [9]:
# -*- coding: utf-8 -*-
"""Agent_class__gSDE.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/Gaianeve/The_real_Fufi/blob/main/Software/Agent_class__gSDE.ipynb

# Agent class 🤖
Defining the the actor-critic NN structure.
"""

# importing libraries
import os
import numpy as np
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torchsummary import summary
import gym
from datetime import datetime


#getting cute unique name for checkpoint
def get_checkpoint_name(epoch_v):
  now = datetime.now()
  today = now.strftime("%Y_%m_%d_%H_%M_%S")
  check_name = 'checkpoint' + '_' + str(epoch_v) + '_' + today
  return check_name

"""## PPO structure 🦄 ✨
Defining the basic layer for PPO
"""

# init layer
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
  torch.nn.init.orthogonal_(layer.weight, std)
  torch.nn.init.constant_(layer.bias, bias_const)
  return layer

"""## Here's the actual agent 🐶 🦾
🪄 Differences from the previous versions:
* Added gSDE:
Mostly taken from stablebaseline implementation [here](https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/distributions.py), called
```
StateDependentNoiseDistribution
```
Paper [here](https://arxiv.org/abs/2005.05719)

* PPO for a continous action space. The code is that of the CleanRL implementation [here](https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/ppo_continuous_action.py).

"""

# agent class
class Agent(nn.Module):
  def __init__(self, envs, use_sde):
      super().__init__()
      #assign environment to interact with
      self.envs = envs
      #gSDE flag
      self.use_sde = use_sde

      #actor critic NN
      self.critic = nn.Sequential(
          layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 1), std=1.0),
      )
      self.actor_mean = nn.Sequential(
          layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, np.prod(envs.single_action_space.shape)), std=0.01),
      )
      self.actor_logstd = nn.Parameter(torch.zeros(1, np.prod(envs.single_action_space.shape)))
      #learn log of standard dev

  ## keep in mind that x are the observations
  def get_value(self, x):
      return self.critic(x)

  def get_action_and_value(self, x, action=None):
      action_mean = self.actor_mean(x)
      action_logstd = self.actor_logstd.expand_as(action_mean) #match dimention of action mean

      if self.use_sde:
        #sample from SDE distribution
        action_dim = np.prod(self.envs.single_action_space.shape)
        int_latent = np.array(self.envs.single_observation_space.shape).prod()
        probs = gSDE(action_dim = action_dim, observation_dim = int_latent, observation = x, mean_actions = action_mean, log_std = action_logstd)
      else:
        #sample from standard gaussian
        action_std = torch.exp(action_logstd)
        probs = Normal(action_mean, action_std)

      if action is None:
          action = probs.sample()
      return action, probs.log_prob(action), probs.entropy(), self.critic(x)

    # NN summary
  def print_summary(self, envs):
    print('Actor summary')
    print(summary(self.actor, envs.single_observation_space.shape))
    print('Critic summary')
    print(summary(self.critic, envs.single_observation_space.shape))

  def get_parameters(self):
    #useful if wanting to check the updating of NN parameters
    for name, param in self.named_parameters():
      print(name, param.data)

   #checkpoints
  def save_checkpoint(self, epoch_v):
      checkpoint_name = get_checkpoint_name(epoch_v)
      directory = os.getcwd() + '/' + 'checkpoints/'
      #if it doesn't exists, then create it
      if not os.path.exists(directory):
          os.mkdir(directory)
          print('Dear human, checkpoint directory did not existed. I created it for you ')
      path = directory + checkpoint_name
      print("=> saving checkpoint '{}'".format(path))
      torch.save(self.state_dict(), path)

  def resume_from_checkpoint(self, path):
      print("=> loading checkpoint '{}'".format(path))
      self.load_state_dict(torch.load(path))

  def save_agent(self, file_name):
      directory = os.getcwd() + '/' + 'models/'
      #if it doesn't exists, then create it
      if not os.path.exists(directory):
          os.mkdir(directory)
          print('Dear human, saved model directory did not existed. I created it for you ')
      path = directory + file_name
      print("=> saving model as best agent in '{}'".format(path))
      torch.save(self.state_dict(), path)

  def load_agent(self, path):
      print("=> loading model from '{}'".format(path))
      self.load_state_dict(torch.load(path))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# -*- coding: utf-8 -*-
"""Agent_utils.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/Gaianeve/FUFONE/blob/main/PPO/Agent_utils.ipynb

# Agent related functions.
"""

#libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time

"""## Annealing
Takes care of annealing, AKA decreasing the learning rate, if requested
"""

#exponentially descrease the learning rate if toggled
def anneal(anneal_lr, update_step, num_update, learning_rate):
  if anneal_lr:
    frac = 1.0 - (update_step - 1.0) / num_update
    lrnow = frac * learning_rate
    return lrnow
  else:
    return learning_rate

"""## Updating agent
Takes care of updating agent relating lists
"""

def collect_data(envs_v, obs_v, actions_v, logprobs_v, rewards_v,\
           dones_v, values_v, next_obs_v, next_done_v, agent_v,\
                 step_loop, device_v):
  obs_v[step_loop] = next_obs_v
  dones_v[step_loop] = next_done_v

  # ALGO LOGIC: action logic
  with torch.no_grad():
      action, logprob, _, value = agent_v.get_action_and_value(next_obs_v)
      values_v[step_loop] = value.flatten()
  actions_v[step_loop] = action
  logprobs_v[step_loop] = logprob

  # TRY NOT TO MODIFY: execute the game and log data.
  next_obs_v, reward, done, info_v = envs_v.step(action.cpu().numpy())
  rewards_v[step_loop] = torch.tensor(reward).to(device_v).view(-1)
  next_obs_v, next_done_v = torch.Tensor(next_obs_v).to(device_v), torch.Tensor(done).to(device_v)

  return obs_v, actions_v, logprobs_v, rewards_v, dones_v, values_v, next_obs_v, next_done_v, info_v

"""## GAE
General Advantage estimation, basically an algorithm to estimate the advantage function.

The original paper about GAE can be found [here](https://arxiv.org/abs/1506.02438)

"""

def GAE(gae_v, gae_lambda_v, gamma_v, agent_v,\
        values_v, dones_v, rewards_v, next_obs_v, next_done_v,\
        num_steps_v, device_v):
  # bootstrap value if not done
  with torch.no_grad():
    next_value = agent_v.get_value(next_obs_v).reshape(1, -1)
    if gae_v:
        advantages_v = torch.zeros_like(rewards_v).to(device_v)
        lastgaelam = 0
        for t in reversed(range(num_steps_v)):
            if t == num_steps_v - 1:
                nextnonterminal = 1.0 - next_done_v
                nextvalues = next_value
            else:
                nextnonterminal = 1.0 - dones_v[t + 1]
                nextvalues = values_v[t + 1]
            delta = rewards_v[t] + gamma_v * nextvalues * nextnonterminal - values_v[t]
            advantages_v[t] = lastgaelam =\
             delta + gamma_v * gae_lambda_v * nextnonterminal * lastgaelam
        returns_v = advantages_v + values_v
    else:
        returns_v = torch.zeros_like(rewards_v).to(device_v)
        for t in reversed(range(num_steps_v)):
            if t == num_steps_v - 1:
                nextnonterminal = 1.0 - next_done_v
                next_return = next_value
            else:
                nextnonterminal = 1.0 - dones_v[t + 1]
                next_return = returns[t + 1]
            returns_v[t] = rewards_v[t] + gamma_v * nextnonterminal * next_return
        advantages_v = returns_v - values_v

  return returns_v, advantages_v

"""## PPO training loop
Training loop with PPO algorithm.

We added kl divergence, that, in a nutshell, it's a simple way to understand how aggressive the policy updates. Further details about the calculation can be found [here](http://joschu.net/blog/kl-approx.html).

"""

def PPO_train_agent(batch_size, update_epochs, minibatch_size, clip_coef, norm_adv, clip_vloss,\
                ent_coef, vf_coef, max_grad_norm, target_kl, \
                agent_v, optimizer_v,scheduler_v, scheduler_flag,\
                b_obs_v, b_actions_v,b_logprobs_v, b_advantages_v, b_returns_v, b_values_v,\
                checkpoint = False):

  #checkpoint is a bool that decides whether to enable or not checkpoint saving

  # Optimizing the policy and value network
  b_inds = np.arange(batch_size)
  clipfracs = []
  loss = 0

  for epoch in range(update_epochs):
      #print('Starting epoch {} of training'.format(epoch))
      np.random.shuffle(b_inds)
      #calculate ratio
      for start in range(0, batch_size, minibatch_size):
          end = start + minibatch_size
          mb_inds = b_inds[start:end]

          _, newlogprob, entropy, newvalue = agent_v.get_action_and_value(b_obs_v[mb_inds],\
                                                                          b_actions_v.long()[mb_inds])
          logratio = newlogprob - b_logprobs_v[mb_inds]
          ratio = logratio.exp()

          # calculate approx_kl
          with torch.no_grad():
              old_approx_kl = (-logratio).mean()
              approx_kl = ((ratio - 1) - logratio).mean()
              clipfracs += [((ratio - 1.0).abs() > clip_coef).float().mean().item()]

          mb_advantages = b_advantages_v[mb_inds]
          if norm_adv:
              mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

          # Policy loss
          pg_loss1 = -mb_advantages * ratio
          pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef)
          pg_loss = torch.max(pg_loss1, pg_loss2).mean()

          # Value loss
          newvalue = newvalue.view(-1)
          if clip_vloss:
              v_loss_unclipped = (newvalue - b_returns_v[mb_inds]) ** 2
              v_clipped = b_values_v[mb_inds] + torch.clamp(
                  newvalue - b_values_v[mb_inds],
                  -clip_coef,
                  clip_coef,
              )
              v_loss_clipped = (v_clipped - b_returns_v[mb_inds]) ** 2
              v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
              v_loss = 0.5 * v_loss_max.mean()
          else:
              v_loss = 0.5 * ((newvalue - b_returns_v[mb_inds]) ** 2).mean()

          entropy_loss = entropy.mean()
          loss_value = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

          #get checkpoints before updating
          if checkpoint:
            if loss_value < loss:
              agent_v.checkpoint(epoch)

          loss = loss_value
          optimizer_v.zero_grad()
          loss.backward()
          nn.utils.clip_grad_norm_(agent_v.parameters(), max_grad_norm)
          optimizer_v.step()
          if scheduler_flag:
            scheduler_v.step(loss_value)

      if target_kl is not None:
          if approx_kl > target_kl:
              break


  return v_loss, pg_loss, entropy_loss, old_approx_kl, approx_kl, clipfracs, b_values_v, b_returns_v

"""## Evaluate the agent

"""


def evaluate_agent(agent_v,gym_id, seed, device, beta,\
                   num_episodes = 10, step_evaluation = 500, eval_with_video = True):

  # make a brand new environment and record the video
  evaluation_video = f'video_evaluation'
  env = vectorize_env(gym_id, seed, eval_with_video, evaluation_video, 1, beta)

  #initialize storage lists
  obs = torch.zeros((step_evaluation, 1) + env.observation_space.shape).to(device)
  actions = torch.zeros((step_evaluation, 1) + env.action_space.shape).to(device)
  logprobs = torch.zeros((step_evaluation, 1)).to(device)
  rewards = torch.zeros((step_evaluation, 1)).to(device)
  dones = torch.zeros((step_evaluation, 1)).to(device)
  values = torch.zeros((step_evaluation, 1)).to(device)

  next_obs = torch.Tensor(env.reset()).to(device)

  # list of episodic returns
  ep_return = []

  for episode in range(0, num_episodes):
    # start the episode
    print('Starting episode %d' %episode, end = '\r')
    for step in range(0, step_evaluation):
      obs[step] = next_obs

      # selecting action with the actor
      with torch.no_grad():
          action, logprob, _, value = agent_v.get_action_and_value(next_obs)
          values[step] = value.flatten()
      actions[step] = action
      logprobs[step] = logprob

      # do the action and getting the rewards
      next_obs, reward, done, info = env.step(action.cpu().numpy())
      rewards[step] = torch.tensor(reward).to(device).view(-1)
      next_obs = torch.tensor(next_obs).to(device)
    # getting the return for the episode
    ep_return.append(sum(rewards))

  # burocracy so that he doesn't complain
  ep_return = torch.tensor(ep_return)
  ep_mean, ep_std = torch.mean(ep_return).item(), torch.std(ep_return).item()
  return ep_mean, ep_std

In [11]:

def parse_args():
    # fmt: off
    parser = argparse.ArgumentParser()
    parser.add_argument("--exp-name", type=str, default= "TheRealMegaFufi_adventures",
        help="the name of this experiment")
    parser.add_argument("--gym-id", type=str, default="PIP-v0",
        help="the id of the gym environment")
    parser.add_argument("--learning-rate", type=float, default=0.00025,
        help="the learning rate of the optimizer")
    parser.add_argument("--seed", type=int, default=1,
        help="seed of the experiment")
    parser.add_argument("--total-timesteps", type=int, default= 50,
        help="total timesteps of the experiments")
    parser.add_argument("--torch-deterministic", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, `torch.backends.cudnn.deterministic=False`")
    parser.add_argument("--cuda", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, cuda will be enabled by default")

    #W&B setup
    parser.add_argument("--track", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="if toggled, this experiment will be tracked with Weights and Biases")
    parser.add_argument("--wandb-project-name", type=str, default="PIPPA",
        help="the wandb's project name")
    parser.add_argument("--wandb-entity", type=str, default=None,
        help="the entity (team) of wandb's project")
    parser.add_argument("--capture-video", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="weather to capture videos of the agent performances (check out `videos` folder)")

    # Algorithm specific hyperparameter
    parser.add_argument("--beta", type=int, default=0.35,
        help="weight of continuity cost")
    parser.add_argument("--num-envs", type=int, default=1,
        help="the number of parallel game environments")
    parser.add_argument("--num-steps", type=int, default=12,
        help="the number of steps to run in each environment per policy rollout")
    parser.add_argument("--anneal-lr", type=lambda x: bool(strtobool(x)), default= True, nargs="?", const=True,
        help="Toggle learning rate annealing for policy and value networks")
    parser.add_argument("--gae", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Use GAE for advantage computation")
    parser.add_argument("--gamma", type=float, default=0.99,
        help="the discount factor gamma")
    parser.add_argument("--gae-lambda", type=float, default=0.95,
        help="the lambda for the general advantage estimation")
    parser.add_argument("--num-minibatches", type=int, default=4,
        help="the number of mini-batches")
    parser.add_argument("--update-epochs", type=int, default=20,
        help="the K epochs to update the policy")
    parser.add_argument("--norm-adv", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles advantages normalization")
    parser.add_argument("--clip-coef", type=float, default=0.2,
        help="the surrogate clipping coefficient")
    parser.add_argument("--clip-vloss", type=lambda x: bool(strtobool(x)), default=True, nargs="?", const=True,
        help="Toggles whether or not to use a clipped loss for the value function, as per the paper.")
    parser.add_argument("--ent-coef", type=float, default=0.01,
        help="coefficient of the entropy")
    parser.add_argument("--vf-coef", type=float, default=0.5,
        help="coefficient of the value function")
    parser.add_argument("--max-grad-norm", type=float, default=0.5,
        help="the maximum norm for the gradient clipping")
    parser.add_argument("--target-kl", type=float, default=None,
        help="the target KL divergence threshold") #should be set to 0.015 if wanna use
    args, unknown = parser.parse_known_args()
    args.batch_size = int(args.num_envs * args.num_steps)
    args.minibatch_size = int(args.batch_size // args.num_minibatches)
    # fmt: on
    return args

In [12]:
args = parse_args()
run_name = f"{args.gym_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

# env setup
envs = vectorize_env(args.gym_id, args.seed, args.capture_video, run_name, args.num_envs, args.beta)
# Agent setup
agent = Agent(envs, use_sde = True).to(device)
#agent.print_summary(envs)
optimizer = optim.Adam(agent.parameters(), lr=args.learning_rate, eps=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'max')

# initializing things
# ALGO Logic: Storage setup
obs = torch.zeros((args.num_steps, args.num_envs) + envs.single_observation_space.shape).to(device)
actions = torch.zeros((args.num_steps, args.num_envs) + envs.single_action_space.shape).to(device)
logprobs = torch.zeros((args.num_steps, args.num_envs)).to(device)
rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
dones = torch.zeros((args.num_steps, args.num_envs)).to(device)
values = torch.zeros((args.num_steps, args.num_envs)).to(device)

## ------------------------------------- START THE GAME -------------------------------------------
# TRY NOT TO MODIFY: start the game
global_step = 0
start_time = time.time()
next_obs = torch.Tensor(envs.reset()).to(device)
next_done = torch.zeros(args.num_envs).to(device)
num_updates = args.total_timesteps // args.batch_size

for update in range(1, num_updates + 1):
  #print('Starting update {}'.format(update))
  # Annealing the rate if instructed to do so.
  optimizer.param_groups[0]["lr"] = anneal(args.anneal_lr, update, num_updates, \
                                            args.learning_rate)

  for step in range(0, args.num_steps):
    # update global steps
    global_step += 1 * args.num_envs
    #update parameters
    obs, actions, logprobs, rewards, dones, values, next_obs, next_done, info \
    = collect_data(envs, obs, actions, logprobs, rewards, dones, values, next_obs,\
                    next_done, agent, step, device)

  # general advantages estimation
  returns, advantages = GAE(args.gae, args.gae_lambda, args.gamma, agent,\
      values, dones, rewards, next_obs, next_done,\
      args.num_steps, device)
  # flatten the batch
  b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
  b_logprobs = logprobs.reshape(-1)
  b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
  b_advantages = advantages.reshape(-1)
  b_returns = returns.reshape(-1)
  b_values = values.reshape(-1)

## ------------------------------------- TRAINING LOOP ----------------------------------------------
  v_loss, pg_loss, entropy_loss, old_approx_kl, approx_kl, clipfracs,\
  b_values, b_returns = PPO_train_agent(args.batch_size, args.update_epochs, args.minibatch_size, \
                                    args.clip_coef, args.norm_adv, args.clip_vloss,\
                                    args.ent_coef, args.vf_coef, args.max_grad_norm, args.target_kl,\
                                    agent, optimizer, scheduler, False,\
                                    b_obs, b_actions,b_logprobs,\
                                    b_advantages, b_returns, b_values)

## --------------------------------- UPDATING AND CLOSING UP -----------------------------------------
  y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
  var_y = np.var(y_true)
  explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y
envs.close()

Dissennatore
Repository Real_Mega_Fufi cloned successfully.
Changed directory to Real_Mega_Fufi/Robot/PIPPA.


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(see

done environment


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:67: DeprecationWarning: WARN: `env.metadata["render.modes"] is marked as deprecated and will be replaced with `env.metadata["render_modes"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment PIP-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:149: DeprecationWarning: WARN: `env.metadata["video.frames_per_second"] is marked as deprecated an

Repository Real_Mega_Fufi cloned successfully.
Changed directory to Real_Mega_Fufi/Robot/PIPPA.
